In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

In [ ]:
tf.random.set_seed(89)

In [ ]:
ds=pd.read_csv("../input/digit-recognizer/train.csv")

In [ ]:
ds.shape

In [ ]:
ds.head()

In [ ]:
X=ds.iloc[:,1:].values
y=ds.iloc[:,0].values

In [ ]:
#normalizing the data
X_scaled=X/255.

In [ ]:
def Network():
    model=tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(784)))
    model.add(tf.keras.layers.Dense(256,activation="relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(128,activation="relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(10,activation="softmax"))
    
    return model
model=Network()

In [ ]:
model.summary()

In [ ]:
from keras import optimizers
optim = optimizers.Adam(lr = 0.0005)
model.compile(loss = 'SparseCategoricalCrossentropy', optimizer = optim, metrics = ['accuracy'])

In [ ]:
# defining scheduler
def scheduler(epoch, lr):
      if epoch < 10:
        return lr
      else:
        return lr * tf.math.exp(-0.1)

In [ ]:
# Bundling all types of callbacks
my_callbacks=([
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=2),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.05,patience=1)
    
    ])

In [ ]:
# Training
trainer = model.fit(X_scaled, y,validation_split=0.3, epochs = 50, batch_size = 32, verbose = 1,callbacks=my_callbacks)

In [ ]:
# Plot loss per iteration
plt.plot(trainer.history['loss'], label='loss')
plt.plot(trainer.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# Plot accuracy per iteration
plt.plot(trainer.history['accuracy'], label='acc')
plt.plot(trainer.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
test=pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
test=test.iloc[:,:].values

In [ ]:
test_scaled=test/255.

In [ ]:
pred = model.predict_classes(test_scaled)

In [ ]:
# create submission file
final_submission = pd.DataFrame({'ImageId': np.array(range(1,28001)), 'Label': pred})
final_submission.to_csv("submission.csv", index=False)

### Just the 1st part
### Next, needed to ensemble different models